# Fake Discord Library
This library is intended to mimic praw so that the book demoes can be run without actually needing reddit credentials, and not actually posting to reddit

In [ ]:
from types import SimpleNamespace

class SimplishNamespace(SimpleNamespace):
    pass

setattr(SimplishNamespace, "__getitem__", lambda self, key: self.__dict__[key])

from IPython.display import HTML, Image, display
import html

import datetime



In [ ]:
def print_info(text, left_margin=0):
    display(
        HTML(
            "<strong style='color:darkred;margin-left:"+str(left_margin)+"px'>" + 
            html.escape(text) + 
            "</strong>"
        )
    )

In [ ]:
print_info("Fake interactions discord is replacing the interactions.py library. Fake discord doesn't need real passwords, and prevents you from accessing real discord")

In [ ]:
async def listen_on_ready():
    print("default onready")
    pass

class FakeOptionType(SimpleNamespace):
    pass

ActualFakeOptionType = FakeOptionType(
        STRING = 1,
        INT = 2)

def fake_listen_builder():
    global listen_on_ready
    def fake_listen(listen_fun):
        global listen_on_ready
        if(listen_fun.__name__ == "on_ready"):
            listen_on_ready = listen_fun

        return lambda: None
    return fake_listen


actions = {}

def fake_slash_command_decorator_builder(name, description, scopes = []):
    def fake_slash_command_decorator(slash_command_fn):
        
        options = []
        if hasattr(slash_command_fn,  "slash_options"):
            options = slash_command_fn.slash_options
        
        actions[name] = {
            "fn": slash_command_fn,
            "options": options
        }
        
        print_info("fake discord interactions setting a slash command called: '" +name + "'.  With options: " + ",".join([option["name"] for option in options]))

        return lambda: None
    return fake_slash_command_decorator



def fake_slash_option_decorator_builder(name, description, required=False, opt_type=ActualFakeOptionType.INT):
    def fake_slash_option_decorator(slash_command_fn):
        # Add slash_options field to function and return it
        if not hasattr(slash_command_fn, "slash_options"):
            slash_command_fn.slash_options = []
        
        slash_command_fn.slash_options.append({
            "name": name,
            "description": description,
            "required": required,
            "opt_type": opt_type
        })
        return slash_command_fn
        
    return fake_slash_option_decorator
        

class FakeSlashContext:
    async def send(self, message):
        print_info("Bot is posting: ")
        print_info(message, 20)
        print_info("")
    

In [ ]:
reply_counter = 0

replies = [
    "jump",
    "do something horrible!",
    "fly",
    "do something horrible!",
    "stop talking. But that doesn't mean I won't say horrible things like: I hate everybody!"
]

In [ ]:
import asyncio
loop = asyncio.get_event_loop()
import nest_asyncio
nest_asyncio.apply()

# TODO Create a client class instead of SimpleNamespace
class FakeDiscordInteractionsClient:
    global reply_counter
    global replies


    def stop(self):
        print_info("Fake discord bot is stopping")


    def start(self):
        global reply_counter
    
        print_info("Fake discord bot is fake logging in and starting to run")
        asyncio.run(listen_on_ready())

        if "do_action" in actions:
            asyncio.run(actions["do_action"]["fn"](FakeSlashContext(), replies[reply_counter]))
            reply_counter += 1

        #test = await self._event.on_ready()
    

def client_creator(token=""):
    print_info("Fake interactions.py is pretending to set up a Discord client connection")
    return FakeDiscordInteractionsClient()

In [ ]:
interactions = SimpleNamespace(
    Client = client_creator,
    listen = fake_listen_builder,
    slash_command = fake_slash_command_decorator_builder,
    slash_option = fake_slash_option_decorator_builder,
    OptionType = ActualFakeOptionType,
    SlashContext = FakeSlashContext
    
    
)